In [2]:

import ee
import os
# import geehydro
# import folium
import numpy as np
import pandas as pd
import datetime
from collections import OrderedDict


## 分割

In [4]:
yDic = [34,29,24,14,1]
path = "./output/106/"
file_x = "_"
Country = ["United Kingdom"]
index = 0 

In [11]:
df = pd.read_excel("centers.xls")
df1 = df.loc[df["SOV0NAME"].isin(Country)][["NAMEASCII","wof_id"]]
name_dic = OrderedDict({})
for i in range(len(df1)):
    city = df1.iloc[i,]["NAMEASCII"]
    fid = int(df1.iloc[i,]["wof_id"])## int not numpy int64
    name_dic[city] = fid
name_dic = OrderedDict(sorted(name_dic.items(), key=lambda t: t[0]))

In [19]:
for i in range(len(name_dic)):
    name = list(name_dic)[i]
    fid = name_dic[name]
    

101750639 Aberdeen
101873467 Ayr
101750387 Bath
101751931 Belfast
1141909171 Birmingham
101750583 Blackpool
101750305 Bournemouth
101750581 Bradford
1141906989 Brighton
101750389 Bristol
101750465 Cambridge
101929817 Cardiff
101750619 Carlisle
101750511 Chester
101750449 Coventry
101872443 Douglas
101854657 Dover
1141906975 Dumfries
101750641 Dundee
101750629 Edinburgh
101750383 Exeter
101872715 Fort William
1141908379 Fox Bay
101938103 George Town
101753853 Gibraltar
101750621 Glasgow
890440113 Grand Turk
101750635 Greenock
1141908377 Grytviken
890442097 Hamilton
101873667 Inverness
101750447 Ipswich
101750561 Kingston upon Hull
101874209 Kirkwall
101750579 Leeds
101750485 Leicester
101874215 Lerwick
1141906971 Lisburn
101750547 Liverpool
101750367 London
101751941 Londonderry
101750401 Luton
101750525 Manchester
101750599 Middlesbrough
101750617 Newcastle
101750483 Norwich
101750491 Nottingham
101873819 Omagh
101750439 Oxford
1141906973 Penzance
1141906969 Perth
101750461 Peterboroug

## test program

In [26]:
def getBandName(center):
    coor = center.getInfo()["coordinates"]
    print("coordinate: ",coor)
    BandLat = np.ceil(coor[1])
    left = "users/zhouzz400/GAIA_2018_lat/GAIA_1985_2018_"
    if BandLat>0:
        Bandname = left+"%02d"% BandLat
        ceilBandname = left+"%02d"% (BandLat+1)
        floorBandname = left+"%02d"% (BandLat-1)
    elif BandLat<-1:
        Bandname = left+"%03d"% BandLat
        ceilBandname = left+"%03d"% (BandLat+1)
        floorBandname = left+"%03d"% (BandLat-1)
    elif BandLat==0:
        Bandname = left+"%02d"% BandLat
        ceilBandname = left+"%02d"% (BandLat+1)
        floorBandname = left+"%03d"% (BandLat-1)
    elif BandLat==-1:
        Bandname = left+"%03d"% BandLat
        ceilBandname = left+"%02d"% (BandLat+1)
        floorBandname = left+"%03d"% (BandLat-1)
    print( Bandname,ceilBandname,floorBandname)
    return Bandname,ceilBandname,floorBandname

In [23]:
import ee
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:10809'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:10809'
ee.Initialize()

In [22]:
name = list(name_dic)[1]
fid = name_dic[name]
name,fid

('Ayr', 101873467)

In [20]:
year_dic = {34:1985,33:1986,32:1987,31:1988,
            30:1989,29:1990,28:1991,27:1992,26:1993,25:1994,24:1995,23:1996,22:1997,21:1998,
            20:1999,19:2000,18:2001,17:2002,16:2003,15:2004,14:2005,13:2006,12:2007,11:2008,
            10:2009, 9:2010, 8:2011, 7:2012, 6:2013, 5:2014, 4:2015, 3:2016, 2:2017, 1:2018,} 

In [24]:
center = ee.Geometry.Point([121.46851393726786, 31.224416065753665])
years=[34,24]
maxdis=30
year = 34

In [40]:
Bandname,ceilBandname,floorBandname = getBandName(center)
GAIA = ee.ImageCollection([Bandname,ceilBandname,floorBandname]).mosaic()

df= pd.DataFrame(columns = ["year","dis","ring_area","water_area","urban_area","dens"])
GAIA_year = GAIA.gte(year)
water = ee.Image("JRC/GSW1_1/YearlyHistory/"+str(year_dic[year])).gte(2)

buffer = ee.List([])
ring = ee.List([])
dis = 1000
buffer = buffer.add( center.buffer(dis) )
ring = ring.add(buffer.get(0))

coordinate:  [121.46851393726786, 31.224416065753665]
users/zhouzz400/GAIA_2018_lat/GAIA_1985_2018_32 users/zhouzz400/GAIA_2018_lat/GAIA_1985_2018_33 users/zhouzz400/GAIA_2018_lat/GAIA_1985_2018_31


In [46]:
ring.getInfo()
ring_urban = GAIA_year.eq(1).clip(ring.get(0))
urban_image = ring_urban.multiply(ee.Image.pixelArea())
urban_area = ee.Number( urban_image.reduceRegion(**{"reducer": ee.Reducer.sum(),"scale": 30,"maxPixels": 1e9}).get("b1") )
urban_area.getInfo()

2634011.66983116

In [ ]:
def getDensity(center):
    Bandname,ceilBandname,floorBandname = getBandName(center)
    GAIA = ee.ImageCollection([Bandname,ceilBandname,floorBandname]).mosaic()
    #Viz_GAIA = {"min": 1, "max": 34, "palette": ['FFFFFF', 'FF0000']}
    # Map.addLayer(GAIA,Viz_GAIA)
    df= pd.DataFrame(columns = ["year","dis","ring_area","water_area","urban_area","dens"])
    for year in years:
        print("year begin: "+str(year_dic[year]))
        GAIA_year = GAIA.gte(year)
        #Viz_year = {"min": 0, "max": 1, "palette": ['FFFFFF', 'FF0000']}
        #Map.addLayer(sh_year,Viz_year,"sh_year")

        water = ee.Image("JRC/GSW1_1/YearlyHistory/"+str(year_dic[year])).gte(2)
        ##water = ee.Image("JRC/GSW1_1/YearlyHistory/2018").gte(2)
        # Viz_water = {"min": 0, "max": 1, "palette": ["ffffff","0000ff"]}
        # Map.addLayer(water,Viz_water,"water")

        # center = ee.Geometry.Point([121.46851393726786, 31.224416065753665])
        buffer = []
        ring = []
        for i in range(maxdis):
            dis = 1000*(i+1)
            buffer.append( center.buffer(dis) )
            if i == 0:
                ring.append(buffer[i])

            else:
                ring.append( buffer[i].symmetricDifference(buffer[i-1]) )

            #Map.addLayer(ring[i])
            ring_urban = GAIA_year.eq(1).clip(ring[i])
            urban_image = ring_urban.multiply(ee.Image.pixelArea())
            urban_area = ee.Number( urban_image.reduceRegion(**{"reducer": ee.Reducer.sum(),"scale": 30,"maxPixels": 1e9}).get("b1") )

            ring_water = water.eq(1).clip(ring[i])
            water_image = ring_water.multiply(ee.Image.pixelArea())
            water_area = ee.Number( water_image.reduceRegion(**{"reducer": ee.Reducer.sum(),"scale": 30,"maxPixels": 1e9}).get("waterClass") )

            ring_area = ring[i].area()
            #print(ring_area.getInfo(),water_area.getInfo(),urban_area.getInfo())
           
            Denominator = ring_area.subtract( water_area )
            dens = urban_area.divide(Denominator).getInfo()
            #print(i,dens)
            
            dic = {"year":str(year_dic[year]),"dis":dis,"ring_area":ring_area.getInfo(),"water_area":water_area.getInfo(),
                   "urban_area":urban_area.getInfo(),"dens":dens}
            df = df.append(dic,ignore_index=True)
            
            if dens<=kdens:
                print("less than "+str(kdens)+" in:"+str(i))
                break
    return  df

In [2]:
df = pd.read_excel("centers.xls")

## 选出特定国家的城市

In [44]:
Countrys = ["China","United States"]
df1 = df.loc[df["SOV0NAME"].isin(Countrys)][["NAMEASCII","wof_id"]]
len(df1)

1178

In [45]:
city = df1.iloc[1,]["NAMEASCII"]

In [46]:
name_dic = {}
for i in range(len(df1)):
    city = df1.iloc[i,]["NAMEASCII"]
    fid = df1.iloc[i,]["wof_id"]
    name_dic[city] = fid

In [47]:
print(name_dic,end = "")

{'Faribault': 85968795, 'Mankato': 85953345, 'Albert Lea': 85969077, 'Willmar': 85953323, 'Brainerd': 85952675, 'Crookston': 85968417, 'Hardin': 85972749, 'Glendive': 85972825, 'Dillon': 85972753, 'Polson': 85972719, 'Devils Lake': 85982371, 'Burley': 85937773, 'Wallace': 1141906267, 'Kennewick': 101729987, 'Centralia': 101730179, 'Glendale': 85917509, 'Safford': 85917391, 'Casa Grande': 85917269, 'Mesa': 85917491, 'Lake Havasu City': 85917415, 'Berkeley': 85921915, 'National City': 85922243, 'Mendocino': 1141906269, 'Paso Robles': 85922313, 'Riverside': 85923345, 'Delano': 85923299, 'San Mateo': 85922431, 'Vallejo': 85922643, 'Glenwood Springs': 85929395, 'Aurora': 85940221, 'Greeley': 85929071, 'Tonopah': 85974973, 'Deming': 85976655, 'Truth or Consequences': 85976585, 'Las Vegas': 85974801, 'Farmington': 85976671, 'Springfield': 85971363, 'Tillamook': 101715925, 'Ontario': 101716235, 'La Grande': 101715803, 'Richfield': 101727989, 'Nephi': 101727673, 'Lander': 101734131, 'Powell': 1

## 查重

一个国家内有没有相同的城市名

In [33]:
df.loc[(df["NAMEASCII"]=="Wuhan") & ( df["SOV0NAME"]=="China")]

,OID,SCALERANK,NATSCALE,LABELRANK,FEATURECLA,NAME,NAMEPAR,NAMEALT,DIFFASCII,NAMEASCII,...,name_it,name_ja,name_ko,name_nl,name_pl,name_sv,name_tr,name_vi,wdid_score,ne_id
7172,7172,2,200,1,Admin-1 capital,Wuhan,,,0,Wuhan,...,Wuhan,武漢市,우한 시,Wuhan,Wuhan,Wuhan,Vuhan,Vũ Hán,4,1159151367


In [37]:
for i in range(len(df)):
    fid = df.iloc[i,]["wof_id"]
    a = len(df.loc[(df["wof_id"]==fid)])
    if a>1 or :
        print(fid)

In [36]:
for i in range(len(df)):
    cityname = df.iloc[i,]["NAMEASCII"]
    counname = df.iloc[i,]["SOV0NAME"]
    a = len(df.loc[(df["NAMEASCII"]==cityname) & ( df["SOV0NAME"]==counname)])
    if a>1:
        print(cityname,counname)

Aurora United States
Las Vegas United States
Springfield United States
Kansas City United States
Longview United States
Pasadena United States
Springfield United States
Salem United States
Montpelier United States
Florence United States
Richmond United States
Lafayette United States
Bowling Green United States
Springfield United States
Lancaster United States
Columbia United States
St. Charles United States
Lancaster United States
Bay City United States
Oktyabrskiy Russia
Dali China
Palmas Brazil
Deer Lake Canada
Mercedes Argentina
Rochester United States
Ely United States
Jamestown United States
Montpelier United States
Longview United States
Aberdeen United States
Pasadena United States
Albany United States
Douglas United States
Fayetteville United States
Burlington United States
St. Charles United States
Columbia United States
Norfolk United States
Aberdeen United States
Huntsville United States
Bay City United States
Albany United States
Columbus United States
Douglas United States

ValueError: too many values to unpack (expected 2)

In [5]:
df.loc[df["NAMEASCII"].isin([ "Bristol"])]["SOV0NAME"]

883     United Kingdom
2054     United States
Name: SOV0NAME, dtype: object

In [10]:
ids =[]
countrys = [ "France","United Kingdom","Japen","China"]
for oid in df.loc[df["SOV0NAME"].isin(countrys)]["OID"]:
    ids.append(str(oid))

In [11]:
len(names)

NameError: name 'names' is not defined

In [12]:
print(ids,end = "")

['880', '881', '882', '883', '884', '885', '886', '887', '888', '908', '909', '910', '911', '912', '1216', '1217', '1218', '1219', '1220', '1221', '1287', '1288', '1289', '1290', '1291', '1292', '1293', '1294', '1295', '1296', '1297', '1298', '1299', '1300', '1301', '1302', '1303', '1304', '1305', '1306', '1307', '1308', '1309', '1312', '1313', '1314', '1315', '1316', '1320', '1321', '1322', '1323', '1324', '1325', '1326', '1327', '1328', '1329', '1330', '1331', '1332', '1333', '1786', '2381', '2382', '2383', '2384', '2385', '2386', '2387', '2388', '2389', '2390', '2391', '2392', '2393', '2394', '2395', '2396', '2397', '2398', '2424', '2425', '2434', '2435', '2436', '2437', '2438', '2439', '2440', '2441', '2442', '2526', '3395', '3396', '3397', '3398', '3399', '3400', '3401', '3402', '3403', '3404', '3405', '3406', '3407', '3408', '3409', '3410', '3411', '3412', '3413', '3414', '3415', '3416', '3417', '3418', '3419', '3420', '3421', '3422', '3423', '3424', '3425', '3426', '3427', '3428

In [16]:
df["SOV0NAME"]

0       Vietnam
1       Vietnam
2       Vietnam
3       Vietnam
4       Vietnam
         ...   
7338    Vietnam
7339    Vietnam
7340    Vietnam
7341    Vietnam
7342    Vietnam
Name: SOV0NAME, Length: 7343, dtype: object